## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import *
import math

## Load Dataset

In [161]:
#load cardio dataset
Cardio = pd.read_excel("~/Downloads/Workout_data.xlsx")

#get index range for different activity types (Generic,Special,Muscle)
gen = Cardio[Cardio.GSM == 1]
G = len(gen)
index_g = G-1

spec = Cardio[Cardio.GSM == 0]
S= len(spec)
index_s = index_g+S

m = Cardio[Cardio.GSM == 2]
M = len(m)
index_m = index_s+M

## User inputs

##### User 1

In [220]:
#1. Choose target 
target = [1,1] #burn calories + muscle building

#2. advanced/beginner
a = 0

#3. weight
Current_weight=57

#4. Number of hours willing to workout per week
w_hours = 5

#5. how many calories user wants to burn per week
Calories = 1000

#6. days of the week preference
Days = [1,1,1,1,1,1,0]

#7. special Activities - day pref.
spec_days=[0,0,0,0,0,1,1]

#8. Category list (if special activity preference)
cat = ["Swimming"]

for i in range(D):
    if (Days[i] == 1 and spec_days[i] == 0):
        avail_days.append(1)
    else:
        avail_days.append(0)

#9.Muscle group preference
Muscle= ["Lower","Upper"]

#10.Muscle mass (in kg) user wants to gain
mm = 1

#if target is only muscle building
if (target[0]==0):
    Calories = 0

#calculate muscle mass gain per week (based on user level)
if (a == 1):
    pw = (Current_weight*(0.005/4))
else:
    pw = (Current_weight*(0.0125/4))

#calculate approximate number of weeks to repeat workout routine for muscle building
n_wks = mm/pw

##### User 2

In [194]:
#1. Choose target 
target = [0,1] #build muscle

#2. advanced/beginner
a = 0

#3. weight
Current_weight=87

#4. Number of hours willing to workout per week
w_hours = 8

#5. how many calories user wants to burn per week
Calories = 0

#6. days of the week preference
Days = [1,1,1,1,1,1,0]

#7. special Activities - day pref.
spec_days=[0,0,0,1,1,1,0]

#8. Category list (if special activity preference)
cat = ["Tennis","Volleyball"]

for i in range(D):
    if (Days[i] == 1 and spec_days[i] == 0):
        avail_days.append(1)
    else:
        avail_days.append(0)

#9.Muscle group preference
Muscle= ["Lower","Core","Upper"]

#10.Muscle mass (in kg) user wants to gain
mm = 3

#if target is only muscle building
if (target[0]==0):
    Calories = 0

#calculate muscle mass gain per week (based on user level)
if (a == 1):
    pw = (Current_weight*(0.005/4))
else:
    pw = (Current_weight*(0.0125/4))

#calculate approximate number of weeks to repeat workout routine for muscle building
n_wks = mm/pw

##### User 3

In [202]:
#1. Choose target 
target = [1,0] #burn calories

#2. advanced/beginner
a = 1

#3. weight
Current_weight=96

#4. Number of hours willing to workout per week
w_hours = 5

#5. how many calories user wants to burn per week
Calories = 3000

#6. days of the week preference
Days = [1,1,1,1,1,1,0]

#7. special Activities - day pref.
spec_days=[0,1,1,0,0,1,1]

#8. Category list (if special activity preference)
cat = []

for i in range(D):
    if (Days[i] == 1 and spec_days[i] == 0):
        avail_days.append(1)
    else:
        avail_days.append(0)

#9.Muscle group preference
Muscle= []

#10.Muscle mass (in kg) user wants to gain
mm = 0

#if target is only muscle building
if (target[0]==0):
    Calories = 0

#calculate muscle mass gain per week (based on user level)
if (a == 1):
    pw = (Current_weight*(0.005/4))
else:
    pw = (Current_weight*(0.0125/4))

#calculate approximate number of weeks to repeat workout routine for muscle building
n_wks = mm/pw

## Pre-processing

In [215]:
import random
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

Cardio_Activities=Cardio['Activity, Exercise or Sport (1 hour)']
Week=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

M = GRB.INFINITY
A=len(Cardio)
D=len(Week)

# Assign random general activities to be chosen (for a more assorted workout)
for i in range(0,G):
    Cardio['Order'][i] = random.randint(0,1)

#Initialise special/general activities picked
Cardio['User_picked'] = 0
        
#Get special activities based on category list
test_s = []
if (len(cat)!=0):
    for i in range(index_g+1,index_s+1):
        if (Cardio['Category'][i] in cat):
            Cardio.at[i,'User_picked'] = 1 #picked
            test_s.append(i)
        else:
            Cardio.at[i,'User_picked'] = 2 #not picked
else:
    for i in range(index_g+1,index_s+1):
        Cardio.at[i,'User_picked'] = 2 #not picked
            
#Get muscle activities based on Muscle list
test_m = []
if (len(cat)!=0):
    for j in range(index_s+1,A):
        c = Cardio.iloc[j]['Category']
        if c in Muscle:
            Cardio.at[j,'User_picked'] = 3 #picked
            test_m.append(j)
        else:
            Cardio.at[j,'User_picked'] = 4 #not picked
else:
    for j in range(index_s+1,A):
        Cardio.at[j,'User_picked'] = 4 #not picked

## Model

In [219]:
model=gb.Model("Workout Plan")
    
#### Decision variables

T=model.addVars(D,A,lb=0,vtype=GRB.INTEGER,name=['T_'+i+'_'+j for i in Week for j in Cardio_Activities])# in 12 minute increments
Z=model.addVars(D,A,vtype=GRB.BINARY,name=['Z_'+i+'_'+j for i in Week for j in Cardio_Activities])
Y=model.addVars(D,A,vtype=GRB.BINARY,name=['Y_'+i+'_'+j for i in Week for j in Cardio_Activities])#dummy variable
X=model.addVars(D,A,vtype=GRB.BINARY,name=['X_'+i+'_'+j for i in Week for j in Cardio_Activities])#dummy variable
W=model.addVars(D,A,vtype=GRB.BINARY,name=['W_'+i+'_'+j for i in Week for j in Cardio_Activities])#dummy variable

#### Objective Function

model.setObjective(sum(T[i,j]*Z[i,j] for i in range(D) for j in range(A)),GRB.MINIMIZE)


#### Constraints

####Common Constraints (apply to both targets)---------------------------------------------------------
#1.1 work out time at most 1 hr per day
for i in range(D):
    model.addConstr(sum(T[i,j] for j in range(A)) <= 5)
    
#1.2 total number of calories: burnt 
model.addConstr(quicksum(T[i,j]*Cardio.iloc[j]["Calories per kg"]*Current_weight for i in range(D) for j in range(A))>=Calories)

#1.3 Workout hours 
model.addConstr(quicksum(T[i,j]*Z[i,j] for i in range(D) for j in range(A))<= w_hours*5)

#1.4 when X>0.01,Z=1,when X<0.01,Z=0,logical constraints
model.addConstrs(10000*Z[i,j]>=T[i,j]-0.01 for i in range(D)for j in range(A))
model.addConstrs(10000*(1-Z[i,j])>=0.01-T[i,j] for i in range(D)for j in range(A))

#1.5 No workout on the days user doesn't want
for i in range(D):
    if (Days[i]==0) and (spec_days[i]==0):
        model.addConstr(sum(Z[i,j] for j in range(A))==0)      
            
#1.6 advanced/beginner constraint
if (a == 1):
    model.addConstr(sum(T[i,j] for i in range(D) for j in Cardio.index[Cardio['Advanced'] == 0].tolist()) == 0)

if (a == 0):
    model.addConstr(sum(T[i,j] for i in range(D) for j in Cardio.index[Cardio['Advanced'] == 1].tolist()) == 0)

#1.7 choose only 1 of special activities
test_s.sort()
if (len(test_s) > 0):
    model.addConstr(sum(Z[i,test_s[x]] for i in range(D) for x in range(len(test_s))) == 1)
    
#1.8 Time constraint for special activity    
if (len(test_s) > 0):
    model.addConstr(sum(T[i,test_s[x]] for i in range(D) for x in range(len(test_s))) == 5)
        
#1.9 no other activities on special activity day        
for i in range(D):
    model.addConstrs(sum(T[i,j] for j in Cardio.index[Cardio['GSM'] == 0].tolist()) <= M*Y[i,j] for j in range(A))
    model.addConstrs(sum(T[i,j] for j in Cardio.index[Cardio['GSM'] != 0].tolist()) <= M*(1-Y[i,j]) for j in range(A))      
        
#1.10 No special activity on normal days
for i in range(D):
    if (spec_days[i]==0):
        model.addConstr(sum(T[i,j] for j in Cardio.index[Cardio['User_picked'] == 1].tolist())==0)

        
####Calorie target Constraints--------------------------------------------------
if (target[0]==1):
    
#2.1 Do not pick from muscle building activities
    if (target[1] == 0):
        for j in range(index_s+1,A):
            model.addConstr(sum(T[i,j] for i in range(D))==0)

#2.2 do not select generic activities where Order = 0 (randomization)
    for j in range(0,G):
         if (Cardio.iloc[j]["Order"] == 0):
                model.addConstr(sum(T[i,j] for i in range(D)) == 0)
                   
#2.3 activities not selected by user should not be picked
    for j in range(A):
        if (Cardio.iloc[j]["User_picked"] == 2):
            model.addConstr(sum(T[i,j] for i in range(D))==0)
            
            
#2.4 each exercise per day should not exceed 24 mins - GENERIC
    for j in range(0,G):
        model.addConstr(sum(T[i,j] for i in range(D))<=2)

        
####Muscle Building Constraints-------------------------------------------------
if (target[1]==1):
    
#3.1 Muscle mass weekly gain constraint
    model.addConstr(quicksum(T[i,j]*Cardio.iloc[j]["Muscle_Mass"]*Current_weight for i in range(D) for j in range(A)) >= pw) 
            
#3.2 Muscle training group not selected by user must not be picked
    for j in range(A):
        if (Cardio.iloc[j]["User_picked"] == 4):
            model.addConstr(sum(T[i,j] for i in range(D))==0)
            
#3.3 Select muscle building exercise + time constraints  
    model.addConstr(sum(T[i,j] for i in range(D) for j in test_m) >= 15) 
    
#3.4 each exercise should be at most 12 mins
    for i in range(D):
        for j in range(index_s+1,A):
            model.addConstr(T[i,j] <= 1)
                        
#3.5 Have at least one activity from each muscle group
    if ("Upper" in Muscle):
        model.addConstr(sum(Z[i,j] for i in range(D) for j in Cardio.index[Cardio['Category'] == 'Upper'].tolist()) >=
                      0.2*sum(Z[i,j] for i in range(D) for j in range(A)))
                        
    if ("Lower" in Muscle):
        model.addConstr(sum(Z[i,j] for i in range(D) for j in Cardio.index[Cardio['Category'] == 'Lower'].tolist()) >=
                       0.2*sum(Z[i,j] for i in range(D) for j in range(A)))
        
    if ("Core" in Muscle and a == 0):
        model.addConstr(sum(Z[i,j] for i in range(D) for j in Cardio.index[Cardio['Category'] == 'Core'].tolist()) >=
                        4)
        
#3.6 if target both, ensure at least one workout for cardio
    if target[0] == 1:
        for i in range (D):
            if (avail_days[i] == 1):
                model.addConstrs(sum(Z[i,j] for j in range(A)) -0.01  <= M*(1-W[i,j]) for j in range (A))
                model.addConstrs(-sum(Z[i,j] for j in range (G)) + 0.01  <= M*W[i,j] for j in range (G))       

model.Params.LogToConsole = 0

model.optimize()

#print objective value
print("Amount of time to workout per week is:", model.objVal/5, " hrs")

#print number of weeks to repeat plan (where applicable)
if (target[1]==1):
    print("Repeat muscle workout plan for around ", math.ceil(n_wks), " weeks")
    
for v in model.getVars():
    if v.x>0:
        print(v.varName,'=',v.x)

Amount of time to workout per week is: 4.4  hrs
Repeat muscle workout plan for around  6  weeks
T_Monday_Bent-Over Dumbbell Row = 1.0
T_Monday_Single-Arm Barbell Curl = 1.0
T_Monday_Push-Up = 1.0
T_Monday_Elevated Hip Lift = 1.0
T_Monday_Dumbbell Lunge = 1.0
T_Tuesday_One Arm Dumbbell Row = 1.0
T_Tuesday_Step-Up = 1.0
T_Wednesday_Pull-Up = 1.0
T_Wednesday_One Arm Dumbbell Row = 1.0
T_Wednesday_Single-Arm Barbell Curl = 1.0
T_Wednesday_Dumbbell Lunge = 1.0
T_Wednesday_Step-Up = 1.0
T_Thursday_Bent-Over Dumbbell Row = 1.0
T_Friday_Close-Grip Lat Pulldown = 1.0
T_Friday_One Arm Dumbbell Row = 1.0
T_Friday_Elevated Hip Lift = 1.0
T_Saturday_Lying Machine Squat = 1.0
T_Sunday_Swimming, slow = 5.0
Z_Monday_Bent-Over Dumbbell Row = 1.0
Z_Monday_Single-Arm Barbell Curl = 1.0
Z_Monday_Push-Up = 1.0
Z_Monday_Elevated Hip Lift = 1.0
Z_Monday_Dumbbell Lunge = 1.0
Z_Tuesday_One Arm Dumbbell Row = 1.0
Z_Tuesday_Step-Up = 1.0
Z_Wednesday_Pull-Up = 1.0
Z_Wednesday_One Arm Dumbbell Row = 1.0
Z_Wednesda